# Tugas Akhir NLP
## Senryuu Shounen
### Tahapan 2: Klasifikasi dengan menggunakan Multinomial Naive Bayes

In [3]:
# Impor modul

import numpy as np
import pandas as pd
import nltk
import sklearn

Class Multinomial Naive Bayes
Kode ini dibuat berdasarkan PR Mata Kuliah Pemelajaran Mesin
Referensi: Andrew NG

In [4]:
class MNB_TextClassifier:
    
    def __init__(self):
        self.prior = {}
        self.condprob = {}
    
    def proba_y_given_x(self, y, x):
        length = len(self.condprob)
        prob_y = self.prior[y]
        prob_x = 1
        for i in x:
            if(i in self.condprob.keys()):
                prob_x *= (self.condprob[i][y])
            else:
                prob_x *= ((1)/(self.dict_y_term[y] + length))
        return prob_y * prob_x
    
    def proba_y(self, y):
        return self.prior[y]
    
    def fit(self, X, y):
        dict_y = {i:list(y).count(i) for i in y}
        total_y = 0

        for i in dict_y:
            total_y += dict_y[i]
        for i in dict_y:
            self.prior[i] = dict_y[i]/total_y
        
        X_Plain = list()
        
        for i in X:
            for j in i:
                X_Plain.append(j)

        dict_x = {i:X_Plain.count(i) for i in X_Plain}
        dict_x_dict_y = {i:dict() for i in dict_x}
        dict_x_cond_y = {i:dict() for i in dict_x}

        for i in range(len(X)):
            for j in X[i]:
                if(i in dict_x_cond_y[j]):
                    dict_x_cond_y[j][i] += 1
                else:
                    dict_x_cond_y[j][i] = 1
        self.dict_y_term = {i:0 for i in y}
        for i in range(len(y)):
            self.dict_y_term[y[i]] += (len(X[i]))

        dict_y_last = {i:{j:0 for j in self.dict_y_term} for i in dict_x_cond_y}

        for i in dict_x_cond_y:
            for j in range(len(y)):
                if(j in dict_x_cond_y[i]):
                    dict_y_last[i][y[j]] += dict_x_cond_y[i][j] 
        self.condprob = {i:{j:0 for j in self.dict_y_term} for i in dict_x_cond_y}
        for i in dict_y_last:
            for j in dict_y_last[i]:

                self.condprob[i][j] = (dict_y_last[i][j] + 1)/(self.dict_y_term[j] + len(dict_x))
        
    def predict_single(self, x):
        classes = [i for i in self.prior.keys()]
        likelihood = [self.proba_y_given_x(i,x) for i in classes]
        return classes[likelihood.index(max(likelihood))]
        
    def predict(self, X):
        return [self.predict_single(x) for x in X]

## Data training
Data training yang digunakan: **single annotator**

In [5]:
single_annotator = pd.read_csv("training_clean/clean_single_annotator.csv",delimiter=",")
single_annotator.head()

,Unnamed: 0,kalimat_id,kata,sense,kalimat
0,0,336691,cerah,4801,cuaca cerah lazim
1,1,336270,cerah,4801,gambar dihasilkan layarnya cerah memiliki spea...
2,2,336555,cerah,4803,cerah pemuda berumur 20 prancis abad 17
3,3,336618,cerah,4801,cor caroli alpha canum venaticorum nama lengka...
4,4,336613,cerah,4801,sanders menyukai cat air lilo maksud menampilk...


In [6]:
df = single_annotator[single_annotator['kata']=='cerah']
arr = [nltk.word_tokenize(df['kalimat'].iloc[i]) for i in range(len(df))]

X = np.array(arr)
y = np.array(df['sense'].values)

clf = MNB_TextClassifier()
clf.fit(X, y)
clf.proba_y_given_x('4801',['cuaca'])
clf.predict_single('cuaca hari ini cerah boyyyyyyy')

'4802'

## Percobaan 1: Testing dengan Data Anotator
Data testing pertama: **double annotator agree**
Token yang digunakan: **cerah**

In [12]:
double_annotator_agree = pd.read_csv("training_clean/clean_double_annotator_agree.csv",delimiter=",")

df_test = double_annotator_agree[double_annotator_agree['kata']=='cerah']

test_arr = [nltk.word_tokenize(df_test['kalimat'].iloc[i]) for i in range(len(df_test))]
test_X = np.array(test_arr)

test_y = np.array(df_test['sense'].values)

y_pred = np.array(clf.predict(test_X))

acc = sklearn.metrics.accuracy_score(test_y, y_pred)

acc

0.8421052631578947

## Percobaan 2: Testing dengan Data Submisi Daring

In [13]:
test_data = pd.read_csv("testing_set/clean_testing_data.csv",delimiter=",")
test_data.head()

,id,word,kalimat
0,13,asing,pecinta film indonesia tv asing mendengar namanya
1,19,asing,telinga asing aneh mendengar menu masakan soto...
2,41,asing,warga negara asing warga negara persemakmuran ...
3,44,asing,belas memerintah sultan mahmud menjalin kerja ...
4,121,asing,diikuti donatdonat waralaba asing master ring ...


In [14]:
words = test_data.word.unique()
words

array(['asing', 'atas', 'badan', 'baru', 'berat', 'besar', 'bidang',
       'bintang', 'bisa', 'buah', 'bulan', 'bunga', 'cabang', 'cerah',
       'coklat', 'dalam', 'dasar', 'dunia', 'halaman', 'harapan', 'jalan',
       'jam', 'jaringan', 'kabur', 'kaki', 'kali', 'kepala', 'ketat',
       'kulit', 'kunci', 'layar', 'lebat', 'lingkungan', 'mata',
       'membawa', 'memecahkan', 'menangkap', 'mendorong', 'menerima',
       'mengandung', 'mengejar', 'mengeluarkan', 'mengikat', 'mengisi',
       'menjaga', 'menurunkan', 'menyusun', 'nilai', 'panas', 'pembagian',
       'rapat', 'sarung', 'tengah', 'tinggi'], dtype=object)

In [15]:
single_annotator = pd.read_csv("training_clean/clean_single_annotator.csv",delimiter=",")
single_annotator.head()

,Unnamed: 0,kalimat_id,kata,sense,kalimat
0,0,336691,cerah,4801,cuaca cerah lazim
1,1,336270,cerah,4801,gambar dihasilkan layarnya cerah memiliki spea...
2,2,336555,cerah,4803,cerah pemuda berumur 20 prancis abad 17
3,3,336618,cerah,4801,cor caroli alpha canum venaticorum nama lengka...
4,4,336613,cerah,4801,sanders menyukai cat air lilo maksud menampilk...


Untuk setiap kata, buat *classifier* MNB dan tulis ke *dataframe* baru.

In [16]:
answer = pd.DataFrame(columns=['kalimat_id', 'kata', 'sense'])
all_rows = []

for word in words:
    df = single_annotator[single_annotator['kata']==word]    
    arr = [nltk.word_tokenize(df['kalimat'].iloc[i]) for i in range(len(df))]
    
    X = np.array(arr)
    y = np.array(df['sense'].values)

    clf = MNB_TextClassifier()
    clf.fit(X, y)
    
    test_current = test_data[test_data['word']==word]
    
    for index, row in test_current.iterrows():
        sense = clf.predict_single(row['kalimat'])
        row_dict = {'kalimat_id':row['id'],
                    'kata':word,
                    'sense':sense}
        all_rows.append(row_dict)

In [17]:
# 5 hasil testing pertama
all_rows[:5]

[{'kalimat_id': 13, 'kata': 'asing', 'sense': '5303'},
 {'kalimat_id': 19, 'kata': 'asing', 'sense': '5303'},
 {'kalimat_id': 41, 'kata': 'asing', 'sense': '5302'},
 {'kalimat_id': 44, 'kata': 'asing', 'sense': '5303'},
 {'kalimat_id': 121, 'kata': 'asing', 'sense': '5303'}]

In [18]:
hasil_df = pd.DataFrame(all_rows,columns=['kalimat_id', 'kata', 'sense'])
hasil_df.head()

,kalimat_id,kata,sense
0,13,asing,5303
1,19,asing,5303
2,41,asing,5302
3,44,asing,5303
4,121,asing,5303


In [19]:
hasil_df.to_csv("hasil_percobaan/hasil_MNB_text_classifier.csv",header=None,index=False)